# part 2 chapter 4
## 머신러닝 회귀 실습

In [45]:
import pandas as pd
train = pd.read_csv("http://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch4/train.csv")
test = pd.read_csv("http://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch4/test.csv")

### EDA

In [46]:
train.shape, test.shape
train.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCR06,12.5,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1,639.1680
1,FDW11,12.6,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1,990.7104
2,FDH32,12.8,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1,2799.6890


In [47]:
train.info(), test.info()
# Object
# train:Item_Identifier, Item_Fat_Content, Item_Type, Outlet_Identifier, Outlet_Size, Outlet_Location_Type, Outlet_Type
# test: Item_Identifier, Item_Fat_Content, Item_Type, Outlet_Identifier, Outlet_Size, Outlet_Location_Type, Outlet_Type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6818 non-null   object 
 1   Item_Weight                5656 non-null   float64
 2   Item_Fat_Content           6818 non-null   object 
 3   Item_Visibility            6818 non-null   float64
 4   Item_Type                  6818 non-null   object 
 5   Item_MRP                   6818 non-null   float64
 6   Outlet_Identifier          6818 non-null   object 
 7   Outlet_Establishment_Year  6818 non-null   int64  
 8   Outlet_Size                4878 non-null   object 
 9   Outlet_Location_Type       6818 non-null   object 
 10  Outlet_Type                6818 non-null   object 
 11  Item_Outlet_Sales          6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIn

(None, None)

In [48]:
train.describe()
test.describe(include = 'O')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,1705,1705,1705,1705,1235,1705,1705
unique,1077,5,16,10,3,3,4
top,FDB56,Low Fat,Fruits and Vegetables,OUT013,Medium,Tier 3,Supermarket Type1
freq,4,997,272,207,565,686,1103


In [49]:
train.isnull().sum()

Item_Identifier                 0
Item_Weight                  1162
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1940
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

### 데이터 전처리

In [50]:
# 인코딩을 할 데이터 타입이 object인 train의 컬럼들을 list로 묶기
cols = list(train.columns[train.dtypes == object])
cols.remove("Item_Identifier")

In [51]:
# target 컬럼을 변수에 옮겨두고 데이터를 합쳐서 인코딩, 인코딩을 위해 train과 test 데이터셋을 병합
target = train.pop('Item_Outlet_Sales')
print(train.shape, test.shape)
df = pd.concat([train, test])
print(df.shape)

(6818, 11) (1705, 11)
(8523, 11)


In [52]:
# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cols:
    df[col] = le.fit_transform(df[col])

In [53]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,NCR06,12.500,1,0.006760,9,42.8112,1,1987,0,2,1
1,FDW11,12.600,1,0.048741,1,60.4194,1,1987,0,2,1
2,FDH32,12.800,1,0.075997,6,97.1410,1,1987,0,2,1
3,FDL52,6.635,2,0.046351,5,37.4506,2,2007,3,1,1
4,FDO09,13.500,2,0.125170,13,261.4910,1,1987,0,2,1


In [55]:
# train과 test를 합쳐서 인코딩했으니 다시 train과 test로 나누기
# train index 번호는 len(train)-1 이고 test index 번호는 len(train)부터 끝까지
train = df.iloc[:len(train)].copy()
test = df.iloc[len(train):].copy()
train.shape, test.shape

((6818, 11), (1705, 11))

In [56]:
# 결측치 채우기
# 결측치를 채울 값은 모델 성능을 보고 바꿔도 괜찮음
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].min())
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

test['Item_Weight'] = test['Item_Weight'].fillna(train['Item_Weight'].min())
test['Outlet_Size'] = test['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

In [58]:
print(train.shape, test.shape)
train.drop('Item_Identifier', axis = 1, inplace = True)
test.drop('Item_Identifier', axis = 1, inplace = True)
print(train.shape, test.shape)

(6818, 11) (1705, 11)
(6818, 10) (1705, 10)


### 검증 데이터 나누기

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((5454, 10), (1364, 10), (5454,), (1364,))

### 머신러닝 학습 및 평가

In [60]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

def rmse(y_val, y_pred):
    return mean_squared_error(y_val, y_pred)**0.5

In [61]:
# 선형 회귀
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)

result = mean_squared_error(y_val, y_pred)
print('mean_squared_error:', result)

result = mean_absolute_error(y_val, y_pred)
print('mean_absolute_error:', result)

result = r2_score(y_val, y_pred)
print('r2_score:', result)

result = rmse(y_val, y_pred)
print('root_mean_squared_error:', result)

mean_squared_error: 1282923.0729833886
mean_absolute_error: 865.1968401416269
r2_score: 0.5058168396924846
root_mean_squared_error: 1132.6619411737063


In [63]:
# RandomForest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

result = mean_squared_error(y_val, y_pred)
print('mean_squared_error:', result)

result = mean_absolute_error(y_val, y_pred)
print('mean_absolute_error:', result)

result = r2_score(y_val, y_pred)
print('r2_score:', result)

result = rmse(y_val, y_pred)
print('root_mean_squared_error:', result)

mean_squared_error: 1102101.12418439
mean_absolute_error: 747.3329991407625
r2_score: 0.5754696224603955
root_mean_squared_error: 1049.8100419525383


In [68]:
# Light_GBM
import lightgbm as lgb
model = lgb.LGBMRegressor(random_state = 0, verbose = -1)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

result = mean_squared_error(y_val, y_pred)
print('mean_squared_error:', result)

result = mean_absolute_error(y_val, y_pred)
print('mean_absolute_error:', result)

result = r2_score(y_val, y_pred)
print('r2_score:', result)

result = rmse(y_val, y_pred)
print('root_mean_squared_error:', result)

mean_squared_error: 1115654.3482227568
mean_absolute_error: 736.6367966578568
r2_score: 0.5702489079618556
root_mean_squared_error: 1056.2454015155554


### 예측 및 결과 파일 생성

In [69]:
pred = model.predict(test)
pred

array([1226.50504175,  830.18920951, 1788.76644309, ..., 3666.34876099,
        990.65125496, 1224.1960178 ], shape=(1705,))

In [71]:
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result_ex4.csv', index=False)